# OPENSTREETMAP CLEANING PROJECT
## By: Gurpal Sandhu

## Introduction - Todo: Split code into 2 parts. Cleaning & Importing, Queries.
This project is designed to show my data cleaning and exploration skills using SQL and python. [I took open-source data from OpenStreetMaps of the Singapore and the area around it](https://www.openstreetmap.org/search?query=singapore#map=11/1.2905/103.8520) to scrub and analyze. 

The steps taken were:
1. Check over the data. Done at first to find shape in python, later to find attribute data to answer questions with in SQL.
2. Clean the data. Focused on cleaning addresses, in particular: street-end names, city names, and postcodes. Started by downloading and cleaning a small portion and then applying to main file. 
3. Convert the data into CSV from an XML formatted .OSM file. Then import into SQL db.
4. Create questions about Singapore I would like to answer, and am able to answer using the data.
5. Answer questions using SQL queries.
6. Gave ideas for improvements to data, or alternate uses.

## Data Cleaning

The data downloaded from OpenStreetMaps is stored in an .osm file, size of 321 MB. An .osm file, as stated by the OpenStreetMaps wikipedia, uses the XML collapsible datastructure, along with a 'type:value' format to keep data tidy. I focused on cleaning the addresses of each node, since the address was what was input by hand by (fallible) human volunteers; in particular, the street-end names, city names, and postcodes.

### Street-End Names

A mapping was used to clean up the different end names. Since I had no awareness of typical Singaporean end names, I had to go back (after exploring with SQL) to include end-names like Lorong and Jalan alongside end-names like Street and Drive.

    MAPPING = {
        'Street':['str', 'st.', 'st'],
        'Road':['rd', 'rd.'],
        'Avenue':['ave', 'av', 'av.', 'ave.'],
        'Jalan':['jl', 'jln', 'jl.'],
        'Lane':['ln', 'ln.'],
        'Drive':['dr', 'dr.'],
        'Lorong':['lr', 'lr.'],
        'Square':['sq', 'sq.']
    }
### Postcodes

Postcodes in Singapore are a 6 number string with the first two numbers the general area, and the last four the exact location. Some online research showed me that the 81 postcode area is the newest postal sector, and in my SQL queries, I noticed that it had many of the bad post codes. The steps used to clean the codes were:
1. Remove all non-numbers. Done using regex (regular expressions module in python).

        newstring = re.sub('[^0-9]','', string)

2. Add 0 in front of 9---- 5 length numbers (there are no codes that start with 9 in Singapore)
3. Add 0 to end of numbers with 2 00s on end that are 5 length numbers (these are estimations)
4. Rest are removed, replaced with ''.

### City Names

Oftentimes, extra information would be put here instead of or alongside the cityname.
First, any city names with any numbers in the string AND with fewer letters than 3 were automatically deleted. Regex was used here.
Then, a mapping was used to fix the rest of the city names. The majority of names are the 5 big ones:

    CITYNAMES = {"singap" : "Singapore",           # singap to encompass all spellings of singap - ur, or, ore, our
                 "johor bahru" : "Johor Bahru",    # written in order of size to choose most likely city for multiple named city
                 "pasir gudang" : "Pasir Gudang", 
                 "batam" : "Batam", 
                 "skudai" : "Skudai"}
There were many variations in the data for city names(ie. Singapore/singapur). As a result, if it had 'singap', 'bahru', 'pasir', 'batam', 'skudai' anywhere in the string, it was changed to align. Finally, the first letter was changed to uppercase - for any city name.

In [72]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

#type = 'addr', key = 'street', 'city', 'postcode', 'housenumber'
# - Clean addresses here, using recompile, mainly postal codes, city names, street end names

#NOTE: filtered renamed cities and estimated postal codes out of print statements, 
#they were the majority of the errors and took up the majority of the print statements

import re

# Seems that 81 is the newest postal sector, has many of the bad postcodes - first 2 digits are sector, last two are 
# If the last two digits of a postcode are '00', it indicates a rough guess of a post code
# how to fix post codes
#remove all non-numbers from postcodes
#add 0 in front of 9---- 5 length numbers (there are no codes that start with 9 in singapore)
#add 0 to end of numbers with 2 00s on end that are 5 length numbers (these are estimations)
#rest should be removed - give examples of the deleted ones

def audit_postcode (string):
    #remove all non-numbers
    newstring = re.sub('[^0-9]','', string)
    if len(newstring) == 6:
        return newstring
    #add 0 in front of 9---- 5 length numbers (there are no codes that start with 9 in singapore, miust be 6 numbers)
    #add 0 to end of numbers with 2 00s on end that are 5 length numbers (these are estimates)
    if len(newstring) == 5:
        if newstring[0] == '9':
            newstring2 = '0' + newstring
            print 'ERROR POSTCODE: ' + newstring + ' -> ' + newstring2
            return newstring2
        if newstring[-2:] == '00':
            newstring3 = newstring + '0'
            print 'ERROR POSTCODE: ' + newstring + ' -> ' + newstring3
            return newstring3
    #rest should be removed - give examples of the deleted ones
    #should we delete all info for the node? nahh
    #we should just delete that line address
    #print 'ERROR POSTCODE: ' + string + ' -> BADSTRING'
    return ''
    
#city names - Singapore, Johor Bahru, Pasir Gudang, Batam, Skudai
#Different Names for main cities - translate into 5 main cities
#check if has 'johor' / 'batam' / 'skudai' / 'singapore' in string - rename to main city using regular expressions
# The problem is writing house#, street name or post code for this part - delete those entries (any that have #s or 
# Make sure to give examples of deleted entries
# There are some that are islands off the coast or non-Johor Malaysia
CITYNAMES = {"singap" : "Singapore",           # singap to encompass all spellings of singap - ur, or, ore, our
             "johor" : "Johor Bahru",    # written in order of size to choose most likely city for multiple named city
             "pasir" : "Pasir Gudang", 
             "batam" : "Batam", 
             "skudai" : "Skudai"}

def audit_cityname (string, citynames = CITYNAMES):
    string = string.lower()
    #check if has 'johor' / 'batam' / 'skudai' / 'singapore' /'pasir' in string - rename to main city using mapping
    for name in citynames.keys():
        if name in string:
            return citynames[name]
    #REMOVE any entries that have numberss in them AND has less than 3 letters in string - show why with list of bad
    if re.search('^#[\d]{2}-[\d]{2}', string):
            print 'ERROR CITY: ' + string + ' -> ADDR:UNIT'
            return 'unit: ' + string 
    if bool(re.search(r'\d', string)) and bool(sum(c.isalpha() for c in string) < 2):
        print 'ERROR CITY: ' + string + ' -> BADSTRING'
        return 'BADSTRING'
    return string.title()  # capitalize any other city names
    
#Street Names - Singapore has many different kinds, hard to fix any
#Make street names consistent with Rd. -> Road, Ave -> Avenue [Search the entire string for inconcistensy]
#Delete anything after designator numbers in string?
#Change any jl, jln, jl. to Jalan (Means way in Malay)
#Recheck data, after each audit -> mayhap delete any string data after 1st comma OR semicolon (incorrect fillout of form) IF it has one of the street endings

MAPPING = {
    'Street':['str', 'st.', 'st'],
    'Road':['rd', 'rd.'],
    'Avenue':['ave', 'av', 'av.', 'ave.'],
    'Jalan':['jl', 'jln', 'jl.'],
    'Lane':['ln', 'ln.'],
    'Drive':['dr', 'dr.'],
    'Lorong':['lr', 'lr.'],
    'Square':['sq', 'sq.']
}

#split string on space AND '.', for each word -> strip spaces:
#convert rd., rd -> Road, jl, jln, jl. -> Jalan, st, st. str -> Street, blvd, dr, ct, pl, ln,
#batu(stone), lorong(hallway), 

#audit_streetname helper function, compares each word to mapping dictionary
def check_word_for_streettype (string, dictionary):
    for streettype, mapping in dictionary.items():
        for abbreviation in mapping:
            if string == abbreviation:
                print 'ERROR STREETNAME: ' + string + ' -> ' + streettype
                return streettype
    return string

def audit_streetname (string):
    audited_addr = ''
    temp_string = string.strip().lower().split()
    for part in temp_string:
        audited_part = check_word_for_streettype(part, MAPPING)
        audited_addr += audited_part.title() + ' '
    return audited_addr[:-1]

def audit_value(value, address_part):
    if address_part == 'postcode':
        return audit_postcode(value)
    elif address_part == 'city':
        return audit_cityname(value)
    elif address_part == 'street':
        return audit_streetname(value)
    return value

## Data Importing / SQL Overview

The map data was extracted from an XML format, and changed from XML to python's dict structure. Then, the values were audited and cleaned as shown above. The data was separated into 5 csv files, and finally imported into SQL under the same headers.  I also tried importing directly from the command line, but the encoding was finicky. The CSV / SQL column names are below.

    NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
    NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
    WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
    WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
    WAY_NODES_FIELDS = ['id', 'node_id', 'position']
### File Sizes
Singapore.osm - 321 MB
nodes, nodes_tags, ways, ways_nodes, ways_tags . csv - 201 MB
SingaporeMap.db - 182 MB

### Node and Ways Count

    sqlite> SELECT COUNT(*) FROM nodes;

    Nodes : 1482437

    sqlite> SELECT COUNT(*) FROM ways;
    
    Ways : 233033
### Unique Contributor Data

    sqlite > SELECT COUNT(DISTINCT(tags.uid))
             FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) AS tags;
             
    Total Contributor Count : 2057

    sqlite > SELECT tags.user, COUNT(*) as num
             FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) AS tags
             GROUP BY tags.user ORDER BY num DESC
             LIMIT 5;
             
    Top 5 Contributors:
        JaLooNz, 395146
        berjaya, 117636
        rene78, 78269
        cboothroyd, 73129
        lmum, 44070

In [75]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

OSM_PATH = "singapore.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def extract_tag_info(element, problem_chars=PROBLEMCHARS):
    tags = []   # Handle secondary tags the same way for both node and way elements
    for tag in element.findall('tag'):
        tag_info = {}
        tag_info['id'] = element.attrib['id']
        tag_info['value'] = tag.attrib['v']
        if problem_chars.match(tag.attrib['k']):
            continue
        elif ':' not in tag.attrib['k']:
            tag_info['key'] = tag.attrib['k']
            tag_info['type'] = 'regular'
        else:
            (tag_info['type'],c,tag_info['key']) = tag.attrib['k'].partition(':')
            if tag_info['type'] == 'addr':
                tag_info['value'] = audit_value(tag_info['value'], tag_info['key'])
                if tag_info['value'] == 'BADSTRING':
                    continue
                if tag_info['value'] == 'addr:unit':
                    tag_info['key'] = 'unit'
                    tag_info['value'] = tag.attrib['v']
        tags.append(tag_info)
    return tags

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    
    if element.tag == 'node':
        for key in node_attr_fields:
            node_attribs[key] = element.attrib[key]
        tags = extract_tag_info(element)
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        for key in way_attr_fields:
            way_attribs[key] = element.attrib[key]
        tags = extract_tag_info(element)
        for position, nd in enumerate(element.findall('nd')):
            way_node_info = {}
            way_node_info['id'] = element.attrib['id']
            way_node_info['node_id'] = nd.attrib['ref']
            way_node_info['position'] = position
            way_nodes.append(way_node_info)
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

# Helper Functions
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# Main Function
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()


        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


process_map(OSM_PATH)


POSSIBLE : sungai rengit
POSSIBLE : sekupang
POSSIBLE : masai
POSSIBLE : masai
POSSIBLE : masai
POSSIBLE : masai
POSSIBLE : masai
POSSIBLE : masai
POSSIBLE : bintan
POSSIBLE : holland village
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : ulu tiram
POSSIBLE : karimun
POSSIBLE : gelang patah
POSSIBLE : punggol
POSSIBLE : changi village
ERROR CITY: #01-58/60 -> ADDR:UNIT
ERROR CITY: #01-50 -> ADDR:UNIT
ERROR CITY: #01-38/40/42 -> ADDR:UNIT
ERROR CITY: #01-46 -> ADDR:UNIT
ERROR CITY: #01-33 -> ADDR:UNIT
ERROR CITY: #01-05 -> ADDR:UNIT
ERROR CITY: #01-06 -> ADDR:UNIT
ERROR CITY: #01-62 -> ADDR:UNIT
ERROR CITY: #01-44 -> ADDR:UNIT
POSSIBLE : sembawang
POSSIBLE : taman bukit dahlia
POSSIBLE : taman perling
POSSIBLE : taman nusantara
POSSIBLE : sembawang
POSSIBLE : sembawang
POSSIBLE : ang mo kio
POSSIBLE : pulai johor
POSSIBLE : 82000
ERROR CITY: 82000 -> BADSTRING
POSSIBLE : sembaw

In [70]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

##function to import any csv file to sqlite3 db, columns_vartype is an OrderedDict with line data
def import_csv(tablename, columns_vartype, filename):

    cursor.execute('''DROP TABLE IF EXISTS {}'''.format(tablename))
    connection.commit()

    # creates strings for CREATE TABLE and INSERT INTO queries of multiple variable amounts
    query1 = ('''CREATE TABLE %s (''' % tablename)
    query2 = ("INSERT INTO %s(" % tablename)
    subquery = "("
    for val in columns_vartype.keys():
        query1 += ('''%s %s NOT NULL''' % (val, columns_vartype[val]))
        if val == columns_vartype.keys()[-1]:
            query1 += (");")
            query2 += ("%s) VALUES " % val)
            subquery += "?);"
        else:
            query1 += (", ")
            query2 += ("%s, " % val)
            subquery += "?, "
    query2 += subquery        
    
    cursor.execute(query1)
    connection.commit()

    
    with open(filename) as fin:
        data = csv.DictReader(fin)
        create_db = []
        for row in data:
            temp_db = []
            for value in columns_vartype.keys():
                temp_db.append(row[value].decode("utf-8"))
            create_db.append(temp_db)
              
    cursor.executemany(query2, create_db)
    connection.commit()

In [76]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

## Import data into SQLITE database
## I tried to import directly from command line, but the encoding was finicky. 
import csv
import sqlite3
from collections import OrderedDict


sqlite3_filename = 'SingaporeMap2.db'
connection = sqlite3.connect(sqlite3_filename)
cursor = connection.cursor()

Nodes_Dictionary = OrderedDict([('id', 'INTEGER'), ('lat', 'REAL'), ('lon', 'REAL'), ('user', 'TEXT'), 
                                ('uid', 'INTEGER'), ('version', 'INTEGER'), ('changeset', 'INTEGER'), ('timestamp', 'TEXT')])
Nodes_Tags_Dictionary = OrderedDict([('id', 'INTEGER'), ('key', 'TEXT'), ('value', 'TEXT'), ('type', 'TEXT')])
Ways_Dictionary = OrderedDict([('id', 'INTEGER'), ('user', 'TEXT'), ('uid', 'INTEGER'), ('version', 'INTEGER'), 
                               ('changeset', 'INTEGER'), ('timestamp', 'TEXT')])
Ways_Nodes_Dictionary = OrderedDict([('id', 'INTEGER'), ('node_id', 'INTEGER'), ('position', 'INTEGER')])
Ways_Tags_Dictionary = OrderedDict([('id', 'INTEGER'), ('key', 'TEXT'), ('value', 'TEXT'), ('type', 'TEXT')])

import_csv('Nodes_Tags', Nodes_Tags_Dictionary, 'nodes_tags.csv')
import_csv('Nodes', Nodes_Dictionary, 'nodes.csv')
import_csv('Ways', Ways_Dictionary, 'ways.csv')
import_csv('Ways_Nodes', Ways_Nodes_Dictionary, 'ways_nodes.csv')
import_csv('Ways_Tags', Ways_Tags_Dictionary, 'ways_tags.csv')

connection.close()

## Answering Questions about Singapore
### Major Streets
I wanted to know what the major streets of Singapore were. I looked for streets with the most OSM objects connected to it.
        
    sqlite > SELECT tags.value, COUNT(*) 
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags 
             WHERE type = 'addr' and key = 'street' 
             GROUP BY tags.value ORDER BY COUNT(*) DESC LIMIT 10;
      
    Joo Chiat Road, 347
    Geylang Road, 261
    Serangoon Road, 246
    Jalan Senang, 234
    South Bridge Road, 197
    Jalan Besar, 192
    North Bridge Road, 188
    Tanjong Pagar Road, 181
    Arab Street, 168
    Westwood Crescent, 165
### Tea vs Coffee Preference
Another question I had was whether a highly developed Eastern city like Singapore prefered coffee, a Western-preferred drink, or tea, an Eastern one.

    sqlite > SELECT COUNT(*) 
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
             WHERE type = 'regular' AND key = 'name' AND tags.value LIKE '% coffee %';
             
    31
    
    sqlite > SELECT COUNT(*) 
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
             WHERE type = 'regular' AND key = 'name' AND tags.value LIKE '% tea %';"
             
    24
### Top Leisure Activities
    sqlite > SELECT tags.value, COUNT(*) as count
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
             WHERE key = 'leisure' GROUP BY tags.value
             ORDER BY count DESC LIMIT 10;
    
    swimming_pool, 1271
    pitch, 1056
    park, 648
    playground, 276
    sports_centre, 110
    park_connector, 83
    fitness_centre, 76
    golf_course, 51
    garden, 48
    recreation_ground', 48
### Top Religions
    sqlite > SELECT tags.value, COUNT(*) as count
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
             WHERE key = 'religion' GROUP BY tags.value
             ORDER BY count DESC LIMIT 10;
    
    muslim, 591
    christian, 245
    buddhist, 106
    hindu, 23
    taoist, 11
    jewish, 4
    sikh, 4
    shinto, 1
### Most Popular Foods
    sqlite > SELECT tags.value, COUNT(*) as count
             FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
             WHERE key = cuisine' GROUP BY tags.value
             ORDER BY count DESC LIMIT 10;
             
    chinese, 170
    burger, 96
    japanese, 86
    pizza, 64
    coffee_shop, 57
    chicken, 55
    asian, 50
    indian, 50
    korean, 49
    italian, 47

## Data Suggestions
Much of the frustrations I had from cleaning the data came from the city names, with many incorrectly spelled major city names - not to mention house numbers, zipcodes, or random information about the location. I believe this was because users are made to input cities manually and are given no extra boxes to put notes on the location. If Open Street Maps had contibutors choose from a drop-down menu to input their city choice, with an 'other' option for extranous situations, much of the headache from inconsistant city naming across the board could be avoided. This would work extremely well if the user's current location is known, or he is inputting information for a major urban center.

I was extremently interested in the 'leisure' tag, to find out what Singaporeans do for fun. Unfortunately, the majority of the data is unusable, with general values such as 'hotel' and 'attraction'. There's an opportunity there for more specific information to be presented.
    
        sqlite > SELECT tags.value, COUNT(*) \
                 FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags
                 WHERE type = 'regular' and key = 'tourism' 
                 GROUP BY tags.value ORDER BY COUNT(*) DESC LIMIT 5;
        
        hotel, 563
        attraction, 274
        hostel, 82
        information, 74
        viewpoint, 59
         

In [129]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

sqlite3_filename = 'SingaporeMap2.db'
connection = sqlite3.connect(sqlite3_filename)
cursor = connection.cursor()

#Finds the streets with the most OSM objects connected to it.
query = "SELECT tags.value, COUNT(*) \
         FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
         WHERE type = 'addr' and key = 'street' \
         GROUP BY tags.value ORDER BY COUNT(*) DESC LIMIT 10;"
#Popular tourist venues
queryA = "SELECT tags.value, COUNT(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
         WHERE type = 'regular' and key = 'tourism' GROUP BY tags.value ORDER BY COUNT(*) DESC;"
#coffee vs tea! - 131 tea, 124 coffee!
queryB = "SELECT COUNT(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
         WHERE type = 'regular' AND key = 'name' AND tags.value LIKE '% coffee %';"
queryC = "SELECT tags.value, COUNT(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
         WHERE type = 'regular' AND key = 'name' AND tags.value LIKE '% tea %' \
         GROUP BY tags.value ORDER BY COUNT(*) DESC;"
#What are the most popular OSM 'objects' created by users? buildings and highways as it turns out
query1 = "SELECT tags.key, COUNT(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
          WHERE type = 'regular' GROUP BY tags.key ORDER BY COUNT(*) DESC"
#most popular non-regular objects (addr, seamarks)
query2 = "SELECT tags.type, COUNT(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags \
          GROUP BY tags.type ORDER BY COUNT(*) DESC"
#City name-likes in the data
query3 = ("SELECT tags.value, COUNT(*) as count "
          "FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) tags "
          "WHERE tags.key LIKE '%city%' "
          "GROUP BY tags.value "
          "ORDER BY count DESC "
          "LIMIT 5;")
#All 'cuisines', 'leisure', 'religion'
query5 = ("SELECT tags.value, COUNT(*) as count "
          "FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) AS tags "
          "WHERE key = 'religion' GROUP BY tags.value \
          ORDER BY count DESC LIMIT 10;"
)
#Use: 
query6 = ("SELECT e.user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e GROUP BY e.user ORDER BY num DESC LIMIT 10;")
for val, row in enumerate(cursor.execute(query5)):
    #if val > 5:
    #    break
    #if (len(row[0]) < 6):
    print row

connection.close()

#Ideas for data discovery
# TEA vs COFFEE shops!
#diverse religions, diverse cuisines, liesure activities, 
#Singapore is known as a meritocracy - heavy emphasis on education - school?, 
#Singapore's government is very dominant in all parts of Singapore - landuse?, military, 
#Singapore's Airport is considered the best in the world. Is there data for it? - aeroway? (get lat, lon -> )
#Singapore prides itself a huge melting pot. Does it live up to that? - check religions and cuisines, denomination, place_of_worship
# type='mtb' is mountain biking info, 'name' gives telugu/english names for things
# type = regular, key = name, shop, foot[footpaths], bicycle, sport(most popular sports), tourism(make map w/ historic?), cuisine, religion, sac_scale(difficulty of hiking routes), 

(u'muslim', 591)
(u'christian', 245)
(u'buddhist', 106)
(u'hindu', 23)
(u'taoist', 11)
(u'jewish', 4)
(u'sikh', 4)
(u'shinto', 1)
